In [ ]:
star1 = [4101289]
star2 = [6819457]
star3 = [7048826]
star4 = [10260828]

In [ ]:
single = category_analysis(blazhko_can, fits, periodogr, ztf_data, dataL, end='test',id_list=star3, plotSave=True)

In [ ]:
ID = 24
Lids = list(FINAL_BE_CATALOGUE['LINEAR id'])

In [ ]:
total_num = len(Lids)
Lid = Lids[ID]
lc_data_ID = dataL.get_light_curve(Lid)
time, mag, magerr = lc_data_ID.T
tZ, mZ, meZ = ztf_data[ID][1], ztf_data[ID][2], ztf_data[ID][3]
for n, j in enumerate(fits):
    if j[0]==Lid:
        break

L1 = fits[n][1][0]
L2 = fits[n][1][2]

fig, ax = plt.subplots(1,4, figsize=(30,5))   
#fig.suptitle('STAR '+str(ID+1)+' from '+str(total_num), fontsize=30, fontproperties=font)
fig.set_facecolor('white')

#--------
ax[0].errorbar(time, mag, magerr, fmt='.k', ecolor=black1,lw=1, ms=4, capsize=1.5,alpha=0.2)
ax[0].set_xlabel('Time (days)',fontproperties=font)
ax[0].set_ylabel('Magnitude (mag)', fontproperties=font)
fig.suptitle('LINEAR object {0}'.format(Lid), fontproperties=font, fontsize=18)
ax[0].invert_yaxis()

# -----

ax[1].set_xlabel('fit LINEAR light curve (best period)', fontproperties=font, fontsize=12)
ax[1].set_ylabel('LINEAR normalized light curve', fontproperties=font, fontsize=12)
ax[1].set_xlim(-0.1, 1.1)
ax[1].set_ylim(1.3, -0.3)
# data
xx, yy, zz = sort3arr(L1['dataPhasedTime'], L1['dataTemplate'], L1['dataTemplateErr'])
ax[1].errorbar(xx, yy, zz, fmt='.k', ecolor=black1, lw=1, ms=4, capsize=1.5, alpha=0.2)
# fit for Plinear
ax[1].plot(L1['modelPhaseGrid'], L1['modTemplate'], or3, markeredgecolor=or3, lw=2, fillstyle='top', linestyle='dashed')

#--------
ax[2].errorbar(tZ, mZ, meZ, fmt='.k', ecolor=black1,lw=1, ms=4, capsize=1.5,alpha=0.2)
ax[2].set_xlabel('Time (days)',fontproperties=font)
ax[2].set_ylabel('Magnitude (mag)', fontproperties=font)
ax[2].invert_yaxis()

# -----

ax[3].set_xlabel('fit ZTF light curve (best period)', fontproperties=font, fontsize=12)
ax[3].set_ylabel('ZTF normalized light curve', fontproperties=font, fontsize=12)
ax[3].set_xlim(-0.1, 1.1)
ax[3].set_ylim(1.3, -0.3)
# data
xx, yy, zz = sort3arr(L2['dataPhasedTime'], L2['dataTemplate'], L2['dataTemplateErr'])
ax[3].errorbar(xx, yy, zz, fmt='.k', ecolor=black1, lw=1, ms=4, capsize=1.5, alpha=0.2)
# fit for Plinear
ax[3].plot(L2['modelPhaseGrid'], L2['modTemplate'], or3, markeredgecolor=or3, lw=2, fillstyle='top', linestyle='dashed')
#plt.tight_layout()
plt.savefig('../img_rsc/lc_pair.png', dpi=650)